In [1]:
from urllib.request import urlopen, Request
from urllib.parse import urlencode
import xml.etree.ElementTree  as ET
from IPython.display import display, HTML
import rasterio as rio

URL = r"https://geo.api.vlaanderen.be/el-dtm/wcs"

In [2]:
ns = {
    'wcs': "http://www.opengis.net/wcs/2.0",
    'ows': "http://www.opengis.net/ows/2.0",
    'gml': "http://www.opengis.net/gml/3.2"
}

In [3]:
req = Request(
    URL + "?request=getcapabilities", 
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    }
)

resp = urlopen(req)
capa = resp.read().decode()

In [5]:
root = ET.fromstring(capa)

for cov in  root.findall('.//wcs:CoverageSummary', namespaces=ns):
    Title = cov.find("./ows:Title", ns)
    covId = cov.find("./wcs:CoverageId", ns) 
    print(Title.text , " -> ", covId.text )

Digitaal Hoogtemodel Vlaanderen II, DTM, raster, 1 m  ->  EL.GridCoverage.DTM


In [ ]:

req = Request(
URL+"?service=WCS&version=2.0.1&request=describecoverage&coverageid=EL.GridCoverage.DTM", 
headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    }
)
resp = urlopen(req)
descript = resp.read().decode()
print(descript)

In [4]:
BBOX = [2.5170092434134, 50.669499282716, 5.9444417082211, 51.51507303755]

params = {
    "SERVICE":"WCS",
    "VERSION":"2.0.1",
    "request":"GetCoverage",
    "CRS":"EPSG:31370",
    "subset":"y,http://www.opengis.net/def/crs/EPSG/0/4258(50.7461,50.9108)",
    "subset":"x,http://www.opengis.net/def/crs/EPSG/0/4258(4.2548,4.5418)",
    "scalefactor":"50",
    "COVERAGEID":"EL.GridCoverage.DTM",
    "FORMAT":"image/tiff", 
    "RESPONSE_CRS": "EPSG:31370"
}

In [25]:
req = Request(
    URL + "?" + urlencode(params), 
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    }
)
print( URL + "?" + urlencode(params) )
resp = urlopen(req)

https://geo.api.vlaanderen.be/el-dtm/wcs?SERVICE=WCS&VERSION=2.0.1&request=GetCoverage&CRS=EPSG%3A31370&subset=x%2Chttp%3A%2F%2Fwww.opengis.net%2Fdef%2Fcrs%2FEPSG%2F0%2F4258%284.2548%2C4.5418%29&scalefactor=50&COVERAGEID=EL.GridCoverage.DTM&FORMAT=image%2Ftiff&RESPONSE_CRS=EPSG%3A31370


In [ ]:
gml = ''
resp.readline() #: first line
for line in resp:
   lined = line.decode(encoding="utf8")
   if '--wcs' in lined: break
   gml +=  lined 

for line in resp:
   lined = line.decode( encoding="ascii")
   if  'Content-Disposition' in lined: break

with open("test.tif", mode="wb") as f:
   for line in resp:
      f.write(line)
print(gml)

In [28]:
ras = rio.open(r'D:\work\geopunt4Qgis\docs\test.tif') 
print(ras.width, ras.height)
ras.close()

5170 1357


In [1]:
import requests


URL = "https://geo.api.vlaanderen.be/el-dtm/wcs"

data_post="""<?xml version="1.0" encoding="UTF-8"?>
<wcs:GetCapabilities
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 xmlns:wcs="http://www.opengis.net/wcs/2.0" 
 xmlns:ows="http://www.opengis.net/ows/2.0" 
 xsi:schemaLocation= "http://schemas.opengis.net/wcs/2.0 ../wcsAll.xsd" service="WCS">
 <ows:AcceptVersions>
 <ows:Version>2.0.1</ows:Version>
 </ows:AcceptVersions>
</wcs:GetCapabilities>"""

headers = {
  'Content-Type': 'application/xml',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0'
}

_r = requests.post(URL, data=data_post, headers=headers)
print( _r.content.decode() )

<?xml version="1.0" encoding="UTF-8"?>
<wcs:Capabilities xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 xmlns:ows="http://www.opengis.net/ows/2.0"
 xmlns:wcs="http://www.opengis.net/wcs/2.0"
 xmlns:gml="http://www.opengis.net/gml/3.2"
 xmlns:crs="http://www.opengis.net/wcs/service-extension/crs/1.0"
 xmlns:int="http://www.opengis.net/WCS_service-extension_interpolation/1.0"
 xmlns="http://www.opengis.net/ows/2.0"
 xmlns:xlink="http://www.w3.org/1999/xlink"
 xsi:schemaLocation="http://www.opengis.net/wcs/2.0 http://schemas.opengis.net/wcs/2.0/wcsAll.xsd" version="2.0.1">
  <ows:ServiceIdentification>
    <ows:Title>el</ows:Title>
  <ows:ServiceType>OGC WCS</ows:ServiceType>
  <ows:ServiceTypeVersion>2.0.1</ows:ServiceTypeVersion>
  <ows:ServiceTypeVersion>1.1.2</ows:ServiceTypeVersion>
  <ows:ServiceTypeVersion>1.1.1</ows:ServiceTypeVersion>
  <ows:ServiceTypeVersion>1.1.0</ows:ServiceTypeVersion>
  <ows:ServiceTypeVersion>1.0.0</ows:ServiceTypeVersion>
  <ows:Profile>http://www

In [8]:
data_post="""<?xml version="1.0" encoding="UTF-8"?> 
<wcs:GetCoverage
 xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
 xmlns:wcs="http://www.opengis.net/wcs/2.0" 
 xmlns:ows="http://www.opengis.net/ows/2.0" 
 xmlns:gml="http://www.opengis.net/gml/3.2" 
 xsi:schemaLocation= "http://schemas.opengis.net/wcs/2.0 ../wcsAll.xsd"
  service="WCS" version="2.0.1"> 
  <wcs:CoverageId>EL.GridCoverage.DTM</wcs:CoverageId> 
  <wcs:trimDimension> 
    <wcs:dimension>x</wcs:dimension> 
    <wcs:trimLow>50.7461</wcs:trimLow> 
    <wcs:trimHigh>50.9108</wcs:trimHigh> 
  </wcs:trimDimension> 
  <wcs:trimDimension> 
    <wcs:dimension>x</wcs:dimension> 
    <wcs:trimLow>4.2548</wcs:trimLow> 
    <wcs:trimHigh>4.5418</wcs:trimHigh> 
  </wcs:trimDimension>
</wcs:GetCoverage>"""

url = 'https://geo.api.vlaanderen.be/el-dtm/wcs'
headers={
    
       'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    }


_r = requests.post(url, data=data_post, headers=headers )
print(_r.text)

<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>Microsoft-Azure-Application-Gateway/v2</center>
</body>
</html>
<!-- a padding to disable MSIE and Chrome friendly error page -->
<!-- a padding to disable MSIE and Chrome friendly error page -->
<!-- a padding to disable MSIE and Chrome friendly error page -->
<!-- a padding to disable MSIE and Chrome friendly error page -->
<!-- a padding to disable MSIE and Chrome friendly error page -->
<!-- a padding to disable MSIE and Chrome friendly error page -->



In [28]:
import rasterio as rio

ras = rio.open(r'WCS:https://geo.api.vlaanderen.be/el-dtm/wcs?VERSION=1.1.0&coverage=EL.GridCoverage.DTM' ) 
ras.profile

{'driver': 'WCS', 'dtype': 'float32', 'nodata': -9999.0, 'width': 258477, 'height': 67839, 'count': 1, 'crs': CRS.from_epsg(4258), 'transform': Affine(-1.3737840783946172e-05, 0.0, 51.559484250526744,
       0.0, 1.3737840783946142e-05, 2.461217912928196), 'blockxsize': 1024, 'blockysize': 512, 'tiled': True}

In [18]:
from osgeo import gdal
url= r'WCS:https://geo.api.vlaanderen.be/el-dtm/wcs?VERSION=1.1.0&coverage=EL.GridCoverage.DTM'
dataset = gdal.Open( url , gdal.GA_ReadOnly)
dataset.GetGeoTransform()

(51.559484250526744,
 -1.3737840783946172e-05,
 0.0,
 2.461217912928196,
 0.0,
 1.3737840783946142e-05)

In [34]:
dataset.RasterXSize//100 , dataset.RasterYSize //100

(2584, 678)

In [36]:
dataset.ReadAsArray(xsize= dataset.RasterXSize//100, ysize= dataset.RasterYSize //100, callback=print)

1.0 Downloading ... None


In [11]:

band = dataset.GetRasterBand(1)
band.ComputeRasterMinMax(True)

scanline = band.ReadRaster(xoff=0, yoff=0,
                        xsize=band.XSize, ysize=1,
                        buf_xsize=band.XSize, buf_ysize=1,
                        buf_type=gdal.GDT_Float32)
scanline

In [8]:
f =[[4,54],[1,2],4,5]
f[1:]

[[1, 2], 4, 5]

In [5]:
import requests

def fetchElevaton(LineString, srs=31370, samples=20 ):
    baseUri = 'https://dhm.agiv.be/api/elevation/v1/DHMV2'
    locations = "|".join( ["{},{}".format(*f) for f in LineString ] )
    data = {}
    data["SrsIn"] = srs
    data["SrsOut"] = srs 
    data["Locations"] = locations
    data["Samples"] = samples
    resp = requests.get(baseUri, params=data )
    return resp.json()

In [6]:
ln = [[152898.7,212780.2],[153071.1,211220.9]]

fetchElevaton(ln)

[[0.0, 152898.7, 212780.2, 6.29],
 [82.56850173779661, 152907.77368421055, 212698.1315789474, 7.25],
 [165.13700347559322, 152916.84736842106, 212616.06315789474, 8.21],
 [247.7055052133898, 152925.92105263157, 212533.9947368421, 8.43],
 [330.27400695118644, 152934.9947368421, 212451.9263157895, 7.93],
 [412.84250868898306, 152944.06842105265, 212369.85789473687, 8.54],
 [495.4110104267797, 152953.14210526316, 212287.7894736842, 8.07],
 [577.9795121645762, 152962.21578947367, 212205.72105263156, 8.69],
 [660.5480139023729, 152971.2894736842, 212123.65263157897, 7.09],
 [743.1165156401695, 152980.36315789475, 212041.58421052631, 7.41],
 [825.6850173779661, 152989.4368421053, 211959.51578947366, 7.56],
 [908.2535191157627, 152998.5105263158, 211877.44736842107, 7.23],
 [990.8220208535594, 153007.58421052631, 211795.37894736842, 7.44],
 [1073.3905225913559, 153016.65789473685, 211713.31052631576, 8.28],
 [1155.9590243291525, 153025.73157894737, 211631.24210526314, 7.4],
 [1238.52752606694